In [1]:
using Flows
using ToySystems
using ToySystems.NineModeSystemEq
using Optim
using PyPlot; pygui(true)
include("9msctrl.jl")
include("Initial condition variation.jl")
include("Perturbation time.jl")
import LinearAlgebra: norm

┌ Info: Recompiling stale cache file C:\Users\Kaijie\.julia\compiled\v1.2\ToySystems\chsON.ji for ToySystems [53c88820-6eff-5a8c-9a74-badd815e7be1]
└ @ Base loading.jl:1240
┌ Warning: Package ToySystems does not have Flows in its dependencies:
│ - If you have ToySystems checked out for development and have
│   added Flows as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ToySystems
└ Loading Flows into ToySystems from project dependency, future warnings for ToySystems are suppressed.


In [2]:
Re_min = 50
Re_step = 1
Re_max = 300
T     = 10000 #or 1000
timestepdifference = 0.5

Re = Re_min:Re_step:Re_max
dadt = zeros(9)
q    = zeros(9)
dqdt = zeros(9)
timearray = timestepdifference:timestepdifference:T
store = RAMStorage(zeros(9))
B = [0,0,0,0,1,0,0,0,0];

In [3]:
function firstel(u::AbstractVector)
    return u[1]
end
function firstel(t, u, dudt, I, dIdt)
    return dIdt[1] = firstel(u)
end
function firstel(xq::Coupled)
    return firstel(xq[1])
end

objfun(u::AbstractVector) = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)
function objfun(t, u, dudt, I, dIdt)
    return dIdt[1] = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)
end
function objfun(xq::Coupled)
    return objfun(xq[1])
end

function Energy_Graph(u0)
    output = zeros(length(Re))
    for i in 1:length(Re)
        u0_local = copy(u0)
        f = NineModeSystem(Re[i])
        ϕ = flow(couple(f,objfun),RK4(couple(zeros(9),zeros(1))), TimeStepConstant(timestepdifference))
        I0 = Float64[0.0]
        ϕ(couple(u0_local, I0), extrema(timearray))
        output[i] = I0[1]/T
    end
#     magnitude = norm(u0)
#     plot(Re, output)
#     xlabel(L"Re")
#     ylabel(L"Objectivefunction")
#     legend(["u0 = $(magnitude)"])
    return output
end

function Energy_Graph_3D(I)
    output = zeros(length(Re))
    for i in 1:length((I[1,:]))
        LengthRe_output = Energy_Graph(I[:,i])
        output = hcat(output,LengthRe_output)
    end
    output = output[:,2:end]
    println(length(output[1,:]))
    return output
end

Energy_Graph_3D (generic function with 1 method)

In [4]:
I = E_Range(E_range)

9×401 Array{Float64,2}:
 0.0  0.0   0.0        0.0        0.0  …  0.0       0.0       0.0       0.0
 0.0  0.05  0.0707107  0.0866025  0.1     0.996243  0.997497  0.998749  1.0
 0.0  0.05  0.0707107  0.0866025  0.1     0.996243  0.997497  0.998749  1.0
 0.0  0.05  0.0707107  0.0866025  0.1     0.996243  0.997497  0.998749  1.0
 0.0  0.05  0.0707107  0.0866025  0.1     0.996243  0.997497  0.998749  1.0
 0.0  0.0   0.0        0.0        0.0  …  0.0       0.0       0.0       0.0
 0.0  0.0   0.0        0.0        0.0     0.0       0.0       0.0       0.0
 0.0  0.0   0.0        0.0        0.0     0.0       0.0       0.0       0.0
 0.0  0.0   0.0        0.0        0.0     0.0       0.0       0.0       0.0

In [6]:
for j in 1:length(I[1,:])
    println(j)
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
